# Identificação

- Discente: Helder Mateus dos Reis Matos
- Matrícula: 202520080003
- Disciplina: Fundamentos de Inteligência Computacional e Ciência de Dados
- Docente: Claudomiro de Souza de Sales Junior
- Tarefa: Atividade 04 - Programas em Python for p-value, confidence intervals e correlation

- Link para o Colab (modo leitura): 

# 0. Configurações

In [ ]:
import itertools
import time
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from scipy.stats import linregress

sns.set_style("whitegrid")

# 1. Correlação

_Faça plotes similares as duas figuras mostradas nos dois próximos slides._

_Cada uma deve ter 9 pontos. A segunda deve ter seus dados gerados aleatoriamente com uma distribuição uniforme no mesmo intervalo da primeira._

_Use a função linregress para achar um reta que represente os dados e também para calcular o p-value. Interprete os resultados. Pesquise sobre a função linregress._

_Calcule a covariância e interprete os resultados._

<div style="display: flex; justify-content: center; gap: 10px;">
    <img src="https://github.com/hellsdeur/ficcd-ppgcc-2025/blob/main/atividade04/imagens/1a.png?raw=true" alt="Figura 1A" width="400px">
    <img src="https://github.com/hellsdeur/ficcd-ppgcc-2025/blob/main/atividade04/imagens/1b.png?raw=true" alt="Figura 1B" width="400px">
</div>

# 2. Estimando um Intervalo de Confiança para a Média

_Obtenha um conjunto de dados amostrais (por exemplo, pesos de indivíduos, notas de testes, etc.)._

_Calcule o intervalo de confiança de 95% para a média da população seguindo os passos:_

- _Calcular a média e o desvio padrão da amostra._
- _Usar a fórmula para intervalo de confiança ou a biblioteca `scipy.stats`._

_Desafio: Repetir o processo para diferentes níveis de confiança (por exemplo, 90%, 99%) e comparar os resultados._

_Visualizar os intervalos de confiança em um gráfico usando matplotlib para mostrar a variabilidade._

# 3. P-value em um Teste de Hipótese

_Forneça dois conjuntos de dados: um para um grupo de controle e outro para um grupo de teste._

- _Formule a hipótese nula._
- _Realizar um teste t de duas amostras usando `scipy.stats.ttest_ind`._

_Interpretar o p-value resultante e decidir se rejeitam a hipótese nula em um  nível de significância dado (por exemplo, 0,05)._

# 4. Significance level

_Faça slides explicando o código abaixo e interpretando os resultados_

https://colab.research.google.com/drive/13V7IT7XbnNVf4VVehObHO6NZUxaqdl_f?usp=sharing

_Teste com dois valores para significance_level_

_Pelo menos quatro slides com a explicação e interpretação._

# 5. Reproduzindo slides em código

Reproduza em Python o conteúdo dos [slides de 24 a 46](https://docs.google.com/presentation/d/1gMKonLnK3buDUGGJGt1AAS-Ki3jT93LZ/edit?usp=sharing&ouid=115043904090702383039&rtpof=true&sd=true).